<a href="https://colab.research.google.com/github/Talha1818/Machine-Learning-Python-Notebooks/blob/master/Text_Generation_with_LSTM_TransData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"talha1818","key":"18ec00ea572d40f57d9d23565ce7cc2b"}'}

In [ ]:
!mkdir /root/.kaggle
!mv kaggle.json /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d aashita/nyt-comments

 96% 460M/480M [00:03<00:00, 204MB/s]
100% 480M/480M [00:03<00:00, 145MB/s]


In [ ]:
!unzip /content/nyt-comments.zip

Archive:  /content/nyt-comments.zip
  inflating: ArticlesApril2017.csv   
  inflating: ArticlesApril2018.csv   
  inflating: ArticlesFeb2017.csv     
  inflating: ArticlesFeb2018.csv     
  inflating: ArticlesJan2017.csv     
  inflating: ArticlesJan2018.csv     
  inflating: ArticlesMarch2017.csv   
  inflating: ArticlesMarch2018.csv   
  inflating: ArticlesMay2017.csv     
  inflating: CommentsApril2017.csv   
  inflating: CommentsApril2018.csv   
  inflating: CommentsFeb2017.csv     
  inflating: CommentsFeb2018.csv     
  inflating: CommentsJan2017.csv     
  inflating: CommentsJan2018.csv     
  inflating: CommentsMarch2017.csv   
  inflating: CommentsMarch2018.csv   
  inflating: CommentsMay2017.csv     


In [ ]:
import pandas as pd
import numpy as np
import string, os
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
# set seeds for reproducability
# from tensorflow import seed
from numpy.random import seed
# set_random_seed(2)
seed(1)
# keras module for building LSTM 
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku

In [ ]:
# Loading the all headlines as a list 
curr_dir = '/content/'
all_headlines = []
for filename in os.listdir(curr_dir):
    if 'Articles' in filename:
        article_df = pd.read_csv(curr_dir + filename)
        all_headlines.extend(list(article_df.headline.values))
        break
all_headlines = [line for line in all_headlines if line!= "Unknown"]
print(all_headlines[:10])

['I Stand  With the ‘She-Devils’', 'Trump’s Birth Control Problems', 'What’s the Craziest Thing You’ve Ever Found in a Xerox Machine?', 'U.S. Allies’ Conflict Is ISIS’ Gain', '$1.5 Trillion Plan on Infrastructure, but Not a Lot of Funding or Details', 'Mueller Zeros In on a Trump Tower Cover Story', ' With Speech, A ‘Dreamers’ Rift Deepens', 'At the Start', '‘The Assassination of Gianni Versace’ Episode 3: Death or Disgrace?', 'Britain’s Model for Outsourcing Services May Be Cracking']


In [ ]:
len(all_headlines)

1066

In [ ]:
CURRENT_FILENAME = 'Text_dataset_27Feb_Merge_with_21Feb'
df = pd.read_csv(f"/content/drive/MyDrive/Annotation Folder/{CURRENT_FILENAME}.csv")

df_two = df[df['target'].isin(['15'])]

df_two['target'] = df_two['target'].map({"15":"MORTAGE"})
df_two.head()

,serial_number,data,state,county,target
0,16070,"HAD DONFIELD - CITY ABSTRACT, L 1 LBS 1 OF 1\n...",NJ,camden,MORTAGE
1,16071,re ‘epee (o| {253 / PNCBAD i c 3-5/310\noe G P...,NJ,camden,MORTAGE
28,16098,"g varanteed a,\n|\nCLOSING AND FUNDING INSTRUC...",NJ,camden,MORTAGE
29,16099,CERTIFIED COPY AND ORIGINAL CLOSING DISCLOSURE...,NJ,camden,MORTAGE
30,16100,failure to properly complete and provide the N...,NJ,camden,MORTAGE


In [ ]:
all_headlines = (df_two['data'] + df_two['target']).tolist()
print(all_headlines[:10])

['HAD DONFIELD - CITY ABSTRACT, L 1 LBS 1 OF 1\nBASH aonon avec Dw 12.10.\nSHIP. TO:\n562401735\n\nGUARANTEED RATE INC\n\n1806 WEST CUYLER AVENUE\n\nSUITE 1W\n\nCHICAGO IL 60613\n\navrn*:| IL 606 9-06\n\n° o =\npy thy\n7 *\nBILLING: P/P\nXOL 22.01.28 NV45 8.04 02/2022* ™\n\x0cMORTAGE', 're ‘epee (o| {253 / PNCBAD i c 3-5/310\noe G PNC Bank, National Association No. ° 0 0 1 3 3 9\nCASHIER’S CHECK _ Philadelphia, PA pATEREBRURRY La Dre?\nPAY TO THE seri neers : . . ny Pre\norpEROF Lilt ABSTRALT, LOE\nTHIRTY OWE THOUSAND FIVE MINDRED TAREE GN) 78 | JRE poe seme\nELIZA MH STAST t ) :\n. Te OREMITTER Os p q VAs ociation\n. 7 Y -_\nMEA\nESC set c101537 , } OFFICIN\nwoooa33sgm 8O3~0000538 850004354 cu S\ni\n\x0cMORTAGE', 'g varanteed a,\n|\nCLOSING AND FUNDING INSTRUCTIONS FROM GUARANTEED RATE, INC. |\n|\nGuaranteed Rate Loan Number 224134165 00 |\nPrimary Borrower Last Name Stasi |\nPrimary Borrower First Name Eliza\nProperty Street Address 332 MARNE AVE 000\nCity HADDONFIELD\nState, Zip NEW

In [ ]:
import re

In [ ]:
def clean_text(txt):
    try:
      txt = "".join(t for t in txt if t not in string.punctuation).lower()
      txt = txt.encode("utf8").decode("ascii",'ignore')
      return txt
    except:
      return ''
corpus = [clean_text(x) for x in all_headlines]
corpus = list(filter(lambda x: x != '', corpus))

# define the regular expression pattern to match digits, newlines, and non-printable characters
pattern = r'[\d\n\x00-\x1f]'
# iterate over the list and remove the matching characters from each string
corpus = [re.sub(pattern, '', s) for s in corpus]
# use a list comprehension to remove extra spaces from each string
corpus = [' '.join(s.split()) for s in corpus]
print(corpus[:10])

['had donfield city abstract l lbs of bash aonon avec dw ship toguaranteed rate inc west cuyler avenuesuite wchicago il avrn il o py thy billing ppxol nv mortage', 're epee o pncbad i c oe g pnc bank national association no cashiers check philadelphia pa paterebrurry la drepay to the seri neers ny preorperof lilt abstralt loethirty owe thousand five mindred taree gn jre poe semeeliza mh stast t te oremitter os p q vas ociation y meaesc set c officinwoooasgm o cu simortage', 'g varanteed aclosing and funding instructions from guaranteed rate inc guaranteed rate loan number primary borrower last name stasi primary borrower first name elizaproperty street address marne ave city haddonfieldstate zip new jersey loan amount document date february loan type conventional closing date eebruary product all fixed loans disbursement february loan term months order number cahdinterest rate escrow number cahd order number jjguaranteed rate closer angela allen phone number fax number email address an

In [ ]:
tokenizer = Tokenizer()
def get_sequence_of_tokens(corpus):
    ## tokenization
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1
    
    ## convert data to a token sequence 
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences, total_words
inp_sequences, total_words = get_sequence_of_tokens(corpus)
print(inp_sequences[:10])

[[1539, 7973], [1539, 7973, 74], [1539, 7973, 74, 93], [1539, 7973, 74, 93, 363], [1539, 7973, 74, 93, 363, 399], [1539, 7973, 74, 93, 363, 399, 3], [1539, 7973, 74, 93, 363, 399, 3, 7974], [1539, 7973, 74, 93, 363, 399, 3, 7974, 7975], [1539, 7973, 74, 93, 363, 399, 3, 7974, 7975, 7976], [1539, 7973, 74, 93, 363, 399, 3, 7974, 7975, 7976, 1645]]


In [ ]:
def generate_padded_sequences(input_sequences):
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
    
    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = ku.to_categorical(label, num_classes=total_words)
    return predictors, label, max_sequence_len

In [ ]:
predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences)

In [ ]:
max_sequence_len

972

In [ ]:
total_words

20377

In [ ]:
def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    # ----------Add Input Embedding Layer
    model.add(Embedding(total_words, 10, input_length=input_len))
    # ----------Add Hidden Layer 1 - LSTM Layer
    model.add(LSTM(100))
    model.add(Dropout(0.1))
    # ----------Add Output Layer
    model.add(Dense(total_words, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model
model = create_model(max_sequence_len, total_words)
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 971, 10)           203770    
                                                                 
 lstm_2 (LSTM)               (None, 100)               44400     
                                                                 
 dropout_2 (Dropout)         (None, 100)               0         
                                                                 
 dense_2 (Dense)             (None, 20377)             2058077   
                                                                 
Total params: 2,306,247
Trainable params: 2,306,247
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(predictors, label, epochs=2, verbose=5)

Epoch 1/2
Epoch 2/2


In [ ]:
def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list],              maxlen=max_sequence_len-1, padding='pre')
        # predicted = model.predict_classes(token_list, verbose=0)
        # predict the classes of input data
        probs = model.predict(token_list, verbose=0)
        predicted = np.argmax(probs, axis=-1)
        # print(predicted)
        
        output_word = ""
        for word,index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " ===>"+output_word
    return seed_text.title()

In [ ]:
print (generate_text("had donfield city abstract l lbs of bash aonon avec dw ship toguaranteed rate inc west cuyler avenuesuite wchicago il avrn il o py thy billing ppxol nv mortage", 1, model, max_sequence_len))
print (generate_text(corpus[150], 1, model, max_sequence_len))
print (generate_text(corpus[34], 1, model, max_sequence_len))
print (generate_text(corpus[200], 1, model, max_sequence_len))
print (generate_text(corpus[500], 1, model, max_sequence_len))
# print (generate_text("science and technology", 1, model, max_sequence_len))

Had Donfield City Abstract L Lbs Of Bash Aonon Avec Dw Ship Toguaranteed Rate Inc West Cuyler Avenuesuite Wchicago Il Avrn Il O Py Thy Billing Ppxol Nv Mortage ===>Mortage
Loan G Closing Feesthe Amounts Provided By Settlement Agent And Disclosed In The Closing Instructions Are Incorrectg Paymentsany Request For A Payment From Loan Proceeds Is Ambiguous Has Missing Or Incomplete Information Lacks A Clear Descriptionor Appears Unusual In Any Mannerg Concessions Incentives Contributions Commissionsa Concession Incentive Contribution Or Commission Which Exceeds The Values Shown In The Specific Closing Instructionswhether Monetary Or Nonmonetary Or Disclosed Or Not Disclosed In Any Sales Contract Or Escrow Instruction Provided Tolenderg Occupancythe Loan Is Shown As Owneroccupied In The Specific Closing Instructions But Borrower Expresses An Intent To Not Occupy Thepropertyclosing Instructions Cius Page Of Bank Of America Naaoe Camasaergerspatek Neeeeemortage ===>Mortage
Loan Additional Ple